In [1]:
from pathlib import Path

import glob
import shutil 
from Bio import SeqIO
import pandas as pd 
import os
import time
import csv

# Initiate functions
from cobra.io import sbml
from chmrapi.io import cobra_model
import cobra


import os
from os.path import join

## Import tables


In [2]:
overview_normLgene = pd.read_csv('./OGcore_Total_overview_Lgene_norm.csv', index_col=0)

overview_normTPM = pd.read_csv('./OGcore_Total_overview_TPM_norm.csv', index_col=0)





## Import model


In [3]:
# Get the currated lac model
data_dir = "/Users/dkinkj/Documents/Azure/CHMR/lactococcus_lactis_sp/model_files/json"

Lac_panmodel = cobra.io.load_json_model(join(data_dir, "panmodel_draft.json"))




In [4]:
print('# Reactions: ', len(Lac_panmodel.reactions))
print('# Metabolites: ', len(Lac_panmodel.metabolites))
print('# Genes: ', len(Lac_panmodel.genes))



# Reactions:  866
# Metabolites:  743
# Genes:  563


In [5]:
Lac_panmodel.solver = 'glpk'
solution = Lac_panmodel.optimize()
solution

,fluxes,reduced_costs
ACALDt,0.000000,-4.250073e-17
ACMANApts,0.000000,0.000000e+00
ACTNdiff,0.000000,0.000000e+00
ACt2r,-8.275635,-0.000000e+00
ADEt2,0.000000,0.000000e+00
...,...,...
URFGTT,0.000000,1.040834e-17
XANt,0.000000,-0.000000e+00
XYLI1,0.000000,0.000000e+00
XYLI2,0.000000,0.000000e+00


In [9]:
gene_in_model = [g.id for g in Lac_panmodel.genes]
len(gene_in_model)

563

## Investigate difference for minus ST

In [7]:
genes_padj_minusST = overview_normLgene[overview_normLgene['padj.minusSTvsAll']<0.01].index
print(len(genes_padj_minusST))

genes_padj_log2FC_minusST = overview_normLgene[(overview_normLgene['padj.minusSTvsAll']<0.01) & (abs(overview_normLgene['log2FoldChange.minusSTvsAll'])>1)].index
len(genes_padj_log2FC_minusST)

368


59

In [10]:
N_gene_in_model_ST=0
sys_change={}
for gene in genes_padj_minusST:
    
    if gene in gene_in_model:
        Lac_panmodel.genes.get_by_id(gene)
        
        #'All_norm: ', round(overview_normLgene.loc[gene]['All_norm'],1), 'minus.5614_norm:', round(overview_normLgene.loc[gene]['minus.5614_norm'],1), 'minus.6086_norm:', round(overview_normLgene.loc[gene]['minus.6086_norm'],1), 'minus.10675_norm:', round(overview_normLgene.loc[gene]['minus.10675_norm'],1), 'minus.SICO_norm:',round(overview_normLgene.loc[gene]['minus.SICO_norm'],1)
        #if overview_normLgene.loc[gene]['padj.minusSTvsAll']<0.01:
        if (overview_normLgene.loc[gene]['padj.minusSTvsAll']<0.01) & (abs(overview_normLgene.loc[gene]['log2FoldChange.minusSTvsAll'])>1):
            N_gene_in_model_ST+=1
        
            print(gene, overview_normLgene.loc[gene]['Description'])
            print('padj.minusSTvsAll', overview_normLgene.loc[gene]['padj.minusSTvsAll'], 'log2FoldChange', round(overview_normLgene.loc[gene]['log2FoldChange.minusSTvsAll'],2))
       
            for r in Lac_panmodel.genes.get_by_id(gene).reactions:
                print(r, 'LB: ', r.lower_bound, 'UB', r.upper_bound, 'flux', r.flux, r.subsystem)
                if r.subsystem ==[]:
                    sys_change['No_subsystem']
                for sys in r.subsystem:
                    if sys in sys_change:
                        sys_change[sys]+=1
                    else:
                        sys_change[sys]=1

N_gene_in_model_ST



OG1286 Ammonium transporter
padj.minusSTvsAll 8.458354250480598e-106 log2FoldChange 4.68
NH4t: nh4_e <=> nh4_c LB:  -1000.0 UB 1000.0 flux 0.0 ['Transport']
OG1305 D-methionine-binding lipoprotein MetQ
padj.minusSTvsAll 6.03472320099067e-25 log2FoldChange -1.63
METabc: atp_c + h2o_c + met__L_e --> adp_c + h_c + met__L_c + pi_c LB:  0.0 UB 1000.0 flux 0.0 ['Exchange', 'Transport']
METDabc: atp_c + h2o_c + met__D_e --> adp_c + h_c + met__D_c + pi_c LB:  0.0 UB 1000.0 flux 0.0 ['Transport']
OG1306 putative D-methionine-binding lipoprotein MetQ
padj.minusSTvsAll 6.509846422671821e-23 log2FoldChange -1.61
METabc: atp_c + h2o_c + met__L_e --> adp_c + h_c + met__L_c + pi_c LB:  0.0 UB 1000.0 flux 0.0 ['Exchange', 'Transport']
METDabc: atp_c + h2o_c + met__D_e --> adp_c + h_c + met__D_c + pi_c LB:  0.0 UB 1000.0 flux 0.0 ['Transport']
OG1307 putative D-methionine-binding lipoprotein MetQ AND D-methionine-binding lipoprotein MetQ
padj.minusSTvsAll 4.4177443775183704e-23 log2FoldChange -1.64
MET

36

In [11]:
sys_change_df=pd.DataFrame.from_dict(sys_change, orient='index')
sys_change_df.columns=['Number_diff_genes']
sys_change_df.sort_values(by='Number_diff_genes', ascending=False)

,Number_diff_genes
Exchange,32
Transport,21
Purine and Pyrimidine Metabolism,18
Nucleotide Metabolism,18
Unassigned,5
Amino Acid Metabolism,3
Folate Metabolism,3
Arginine and Proline Metabolism,2
One Carbon pool by folate,2
Nitrogen Metabolism,1


In [12]:
minusST_log2FoldChange = overview_normLgene[['log2FoldChange.minusSTvsAll']]

minus5614_log2FoldChange = overview_normLgene[['log2FoldChange.minus5614vsAll']]
minus6086_log2FoldChange = overview_normLgene[['log2FoldChange.minus6086vsAll']]
minus10675_log2FoldChange = overview_normLgene[['log2FoldChange.minus10675vsAll']]
minusSICO_log2FoldChange = overview_normLgene[['log2FoldChange.minusSICOvsAll']]


In [13]:
minusST_log2FoldChange.to_csv('./DataEscher/DataEscher_core_minusST_log2FoldChange.csv')
minus5614_log2FoldChange.to_csv('./DataEscher/DataEscher_core_minus5614_log2FoldChange.csv')
minus6086_log2FoldChange.to_csv('./DataEscher/DataEscher_core_minus6086_log2FoldChange.csv')
minus10675_log2FoldChange.to_csv('./DataEscher/DataEscher_core_minus10675_log2FoldChange.csv')
minusSICO_log2FoldChange.to_csv('./DataEscher/DataEscher_core_minusSICO_log2FoldChange.csv')


